In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

＃透過 Vertex AI Gemini API 和 Python SDK 呼叫函式

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/gemini/function-calling/intro_function_calling.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fdoggy8088%2Fgenerative-ai%2Fmain%2Fgemini%2Ffunction-calling%2Fintro_function_calling.zh.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise 標誌"><br> 在 Colab Enterprise 中執行
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/gemini/function-calling/intro_function_calling.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/main/gemini/function-calling/intro_function_calling.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
| 作者 | [Kristopher Overholt](https://github.com/koverholt) |


## 總覽

### Gemini

Gemini 是一系列由 Google DeepMind 研發的生成式 AI 模型，旨在用於多模態案例。

### 從 Gemini 呼叫函式

[函式呼叫](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) 讓開發人員能在他們的程式碼中建立函式描述，再將該描述傳遞給請求中的語言模型。模型的回應會包括符合描述的函式名稱，以及用來呼叫函式的引數。

函式呼叫類似於 [Vertex AI Extensions](https://cloud.google.com/vertex-ai/docs/generative-ai/extensions/overview) ，因為它們都產生函式的相關資訊。兩者之間的差異在於，函式呼叫會傳回包含函式名稱及程式碼中使用的引數的 JSON 資料，而 Vertex AI Extensions 會傳回函式並為你呼叫函式。


### 目標

在本教學課程中，你會瞭解如何使用 Vertex AI SDK for Python 搭配 Vertex AI Gemini API，透過 Gemini 1.0 Pro (`gemini-1.0-pro`) 模型來進行函式呼叫。

你將完成下列任務：

- 安裝 Vertex AI SDK for Python
- 使用 Vertex AI Gemini API 與 Gemini 1.0 Pro (`gemini-1.0-pro`) 模型進行互動：
    - 從文字提示產生函式呼叫，以取得特定地點的天氣資訊
    - 從文字提示產生函式呼叫並呼叫外部 API 來對地址進行地理編碼
    - 從聊天提示產生函式呼叫，以協助零售使用者


### 成本

本教學課程使用 Google Cloud 的可計費元件：

- Vertex AI

瞭解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)，並使用 [定價計算器](https://cloud.google.com/products/calculator/) 根據預計使用量產生成本估計。


## 開始使用


### 安裝 Vertex AI SDK for Python


In [ ]:
!pip3 install --upgrade --user google-cloud-aiplatform

### 重新啟動目前的執行階段

要在此 Jupyter 執行階段中使用新安裝的套件，你必須重新啟動執行階段。你可以執行下列Cell來執行此項操作，如此將重新啟動目前的Kernel。


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ Kernel將重新啟動。請等待它完成，再繼續執行下一個步驟。⚠️</b>
</div>


### 驗證你的筆記本環境 (僅針對 Colab)

如果你使用 Google Colab 執行這個筆記本，執行以下Cell以驗證你的環境。如果使用 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)，則不需要執行此步驟。


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 定義 Google Cloud 專案資訊及初始化 Vertex AI

針對你的專案初始化 Python 版的 Vertex AI SDK：


In [2]:
# Define project information
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### 匯入函式庫


In [3]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

## 程式碼範例

### 為何要呼叫函式？

在與生成文字模型共事時，可能難以強制 LLM 給出結構化的格式，例如 JSON，且能提供一致的回應。函式呼叫讓你可以透過提示和非結構化輸入輕鬆地與 LLM 共處，並讓 LLM 回傳一個結構化的回應，可用於呼叫外部函式。

你可以將函式呼叫視為一種從使用者的提示和函式定義中取得結構化輸出、使用結構化輸出對外部系統做出 API 要求，然後將函式回應回傳至 LLM 以產生回應的方法。換句話說，Gemini 中的函式呼叫會從非結構化的文字或使用者的訊息中萃取結構化參數。


### 使用 Gemini 1.0 Pro 模型

Gemini 1.0 Pro (`gemini-1.0-pro`) 模型可處理自然語言任務、多輪文本和程式碼聊天及程式碼生成。


In [4]:
model = GenerativeModel("gemini-1.0-pro")

### 簡單函式呼叫範例


要開始，你將使用函式呼叫設定天氣 API 要求，讓使用者取得給定位置的目前情況。函式參數以 Python 字典指定，並符合 [OpenAPI JSON schema 格式](https://spec.openapis.org/oas/v3.0.3#schemawr)。

考慮一個範例天氣 API 函式，函式中有一個使用者的位置參數，如下所示：

python
def get_current_weather(location):
    ...


你將從指定函式宣告和參數開始，以便對範例天氣 API 提出請求：


In [5]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

然後你可以定義 LLM 要呼叫的工具，其中包含` get_current_weather_func `：


In [6]:
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

接著即可讓模型產生內容，包括「工具」這項你剛剛建置的，以產生回應：


In [7]:
prompt = "What is the weather like in Boston?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)

response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 23
  candidates_token_count: 7
  total_token_count: 30
}

可以檢視回應的功能呼叫部分：


In [8]:
response.candidates[0].content.parts[0].function_call

name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "Boston"
    }
  }
}

回應中包括一個函式簽章，該函式簽章可用於呼叫天氣 API。在這個時候，你擁有形成請求內文和對外系統進行 API 呼叫所需的一切。做得好！


### 複雜函式呼叫範例


在這個範例中，你將產生結構較為複雜的函式呼叫。你會使用函式回應建立 API 呼叫，以將地址轉換為緯度和經度座標。

先在工具內定義一個函式宣告：


In [9]:
get_location = FunctionDeclaration(
    name="get_location",
    description="Get latitude and longitude for a given location",
    parameters={
        "type": "object",
        "properties": {
            "poi": {"type": "string", "description": "Point of interest"},
            "street": {"type": "string", "description": "Street name"},
            "city": {"type": "string", "description": "City name"},
            "county": {"type": "string", "description": "County name"},
            "state": {"type": "string", "description": "State name"},
            "country": {"type": "string", "description": "Country name"},
            "postal_code": {"type": "string", "description": "Postal code"},
        },
    },
)

location_tool = Tool(
    function_declarations=[get_location],
)

現在你可以呼叫模型來產生一個回應：


In [10]:
prompt = """
I want to get the lat/lon coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043, US
"""

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[location_tool],
)
response.candidates[0].content.parts[0]

function_call {
  name: "get_location"
  args {
    fields {
      key: "city"
      value {
        string_value: "Mountain View"
      }
    }
    fields {
      key: "country"
      value {
        string_value: "US"
      }
    }
    fields {
      key: "postal_code"
      value {
        string_value: "94043"
      }
    }
    fields {
      key: "state"
      value {
        string_value: "CA"
      }
    }
    fields {
      key: "street"
      value {
        string_value: "1600 Amphitheatre Pkwy"
      }
    }
  }
}

你現在可以從函式回傳內容中提取結果並建立 API 呼叫：


In [11]:
x = response.candidates[0].content.parts[0].function_call.args

url = "https://nominatim.openstreetmap.org/search?"
for i in x:
    url += '{}="{}"&'.format(i, x[i])
url += "format=json"

x = requests.get(url)
content = x.json()
content

[{'place_id': 377680635,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 2192620021,
  'lat': '37.4217636',
  'lon': '-122.084614',
  'class': 'office',
  'type': 'it',
  'place_rank': 30,
  'importance': 0.6949356759210291,
  'addresstype': 'office',
  'name': 'Google Headquarters',
  'display_name': 'Google Headquarters, 1600, Amphitheatre Parkway, Mountain View, Santa Clara County, California, 94043, United States',
  'boundingbox': ['37.4217136', '37.4218136', '-122.0846640', '-122.0845640']}]

做得好！你能夠構造一個函式和工具讓 LLM 用於輸出函式呼叫所需參數，然後你實際上執行了該函式呼叫以取得指定位置的座標。

我們在此使用 [OpenStreetMap Nominatim API](https://nominatim.openstreetmap.org/ui/search.html) 將地址進行地理編碼，以便於在本筆記本中使用和學習。如果你處理大量地圖或地理定位資料，可以使用 [Google Maps 地理編碼 API](https://developers.google.com/maps/documentation/geocoding)。


### 對話式運算中函式呼叫


在此範例中，你將使用 Gemini 中的聊天模型協助客戶取得商店中商品的資訊。

你首先將在取得商品資訊、取得商店位置和下訂單的工具中定義多個函式：


In [12]:
get_product_info_func = FunctionDeclaration(
    name="get_product_sku",
    description="Get the SKU for a product",
    parameters={
        "type": "object",
        "properties": {
            "product_name": {"type": "string", "description": "Product name"}
        },
    },
)

get_store_location_func = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

place_order_func = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "object",
        "properties": {
            "product": {"type": "string", "description": "Product name"},
            "address": {"type": "string", "description": "Shipping address"},
        },
    },
)

retail_tool = Tool(
    function_declarations=[
        get_product_info_func,
        get_store_location_func,
        place_order_func,
    ],
)

請注意，你還可以在多輪聊天中使用函式呼叫，並且你可以在建立模型時指定工具，避免每次要求都必須傳送工具：


In [13]:
model = GenerativeModel(
    "gemini-1.0-pro", generation_config={"temperature": 0}, tools=[retail_tool]
)
chat = model.start_chat()

好！讓我們開始對話，詢問他們是否有某個產品庫存：


In [14]:
prompt = """
Do you have the Pixel 8 Pro in stock?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_sku"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

預期中，回應包含一個結構化函式呼叫，用於與外部系統進行溝通。

在真實情況中，你會對外部系統或資料庫執行函式呼叫。由於這個筆記本著重在萃取函式參數和產生函式呼叫的能力，你會使用模擬資料回傳給模型，而不是使用真正的 API 伺服器。


In [15]:
# This is where you would make an API request to return the status of their order.
# Use synthetic data to simulate a response payload from an external API.

api_response = {"sku": "GA04834-US", "in_stock": "yes"}

現在，我們將包括外部 API 呼叫的詳細資訊併為使用者產生一個回應：


In [16]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_sku",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, we have the Pixel 8 Pro in stock. The SKU is GA04834-US."

接下來，使用者可能會詢問他們可以在附近商店購買哪一種類型的電話：


In [17]:
prompt = """
Is there a store in Mountain View, CA that I can visit to try it out?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

我們透過另一個結構化函式呼叫取得一個回應，這次設定為使用我們工具中的不同函式。


In [18]:
# This is where you would make an API request to get the location of the store closest to the user.
# Use synthetic data to simulate a response payload from an external API.

api_response = {"store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"}

再一次，我們可以加入外部 API 呼叫的細節，並對使用者生成回應：


In [19]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, there is a store in Mountain View, CA that you can visit to try out the Pixel 8 Pro. The address is 2000 N Shoreline Blvd, Mountain View, CA 94043, US."

最後，使用者可能要求訂購一部手機，並將其運送到某個地址：


In [20]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "place_order"
  args {
    fields {
      key: "address"
      value {
        string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
      }
    }
    fields {
      key: "product"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

完美！我們提取了使用者的產品以及地址，現在我們可以呼叫 API 來下訂單：


In [21]:
# This is where you would make an API request to return the status of their order.
# Use synthetic data to simulate a response payload from an external API.

api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

如前所述，我們將來自外部 API 呼叫的詳細資訊加入並產生對使用者的回應：


In [22]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

text: "OK. I have placed an order for a Pixel 8 Pro and it will be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089. The estimated arrival time is 2 days. Your order number is 12345."

完成！你已成功與 Gemini 進行多回合對話，產生功能呼叫，處理將 (模擬) 資料傳回模型，以及產生使用功能回應的訊息。
